In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import sklearn.metrics as metrics
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Use tensorflow.keras.preprocessing.image to import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16

In [4]:
num_classes=3
IMAGE_SHAPE = [224, 224]
batch_size=32 #change for better accuracy based on your dataset
epochs = 5 #change for better accuracy based on your dataset

In [5]:
vgg = VGG16(input_shape = (224,224,3), weights = 'imagenet', include_top = False)
for layer in vgg.layers:
  layer.trainable = False
x = Flatten()(vgg.output)
x = Dense(128, activation ="relu")(x) 
x = Dense(64, activation = "relu")(x) 
x = Dense(num_classes, activation = "softmax")(x) 
model = Model(inputs = vgg.input, outputs = x)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [6]:
trdata = ImageDataGenerator()
# Ensure the path to your training data is correct
train_data_gen = trdata.flow_from_directory(directory="./Train",target_size=(224,224), shuffle=False, class_mode="categorical") 
tsdata = ImageDataGenerator()
# Ensure the path to your test data is correct
test_data_gen = tsdata.flow_from_directory(directory="./Test", target_size=(224,224),shuffle=False, class_mode="categorical")

Found 385 images belonging to 3 classes.
Found 138 images belonging to 3 classes.


In [9]:
training_steps_per_epoch = int(np.ceil(train_data_gen.n / batch_size)) 
validation_steps_per_epoch = int(np.ceil(test_data_gen.n / batch_size)) 
model.fit(train_data_gen, validation_data=test_data_gen,epochs=epochs, verbose=1) # Add the missing closing parenthesis
print("Training Completed!")

Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.4179 - loss: 16.1653 - val_accuracy: 0.7319 - val_loss: 8.9674
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.8362 - loss: 3.6590 - val_accuracy: 0.8116 - val_loss: 4.0748
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.8697 - loss: 1.9525 - val_accuracy: 0.8333 - val_loss: 4.7250
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.9797 - loss: 0.1934 - val_accuracy: 0.8841 - val_loss: 2.9693
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step - accuracy: 0.9982 - loss: 0.0051 - val_accuracy: 0.8913 - val_loss: 3.4445
Training Completed!


In [10]:
Y_pred = model.predict(test_data_gen, test_data_gen.samples / batch_size)
val_preds = np.argmax(Y_pred, axis=1)
import sklearn.metrics as metrics
val_trues =test_data_gen.classes
from sklearn.metrics import classification_report
print(classification_report(val_trues, val_preds))

5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step
              precision    recall  f1-score   support

           0       0.98      0.83      0.90        59
           1       0.91      0.96      0.94        53
           2       0.72      0.88      0.79        26

    accuracy                           0.89       138
   macro avg       0.87      0.89      0.88       138
weighted avg       0.90      0.89      0.89       138



In [11]:
Y_pred = model.predict(test_data_gen, test_data_gen.samples / batch_size)
val_preds = np.argmax(Y_pred, axis=1)
val_trues =test_data_gen.classes
cm = metrics.confusion_matrix(val_trues, val_preds)
cm

5/5 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step


array([[49,  3,  7],
       [ 0, 51,  2],
       [ 1,  2, 23]], dtype=int64)

In [12]:
keras_file="Model.h5"
tf.keras.models.save_model(model,keras_file)